# Part 1

In [1]:
filename, row = "inputs/12-15.txt", 2000000
with open(filename, "r") as f:
    data = f.read()

In [2]:
import re

In [3]:
sensors = []
for l in data.splitlines():
    x, y, bx, by = (int(v) for v in re.findall("-?\d+", l))
    d = abs(bx -x) + abs(by - y)
    sensors.append((x, y, bx, by, d))

In [4]:
def add_range(ranges, range_):
    x, y = range_
    ret = []
    for (rx, ry) in ranges:
        if rx > y or x > ry:
            ret.append((rx, ry))
        else:
            x = min(x, rx)
            y = max(y, ry)
    ret.append((x, y))
    return ret

In [5]:
ranges = []
row_stations = set()
for x, y, bx, by, d in sensors:
    for s in ((x, y), (by, by)):
        if s[1] == row:
            row_stations.add(s)
    x_range = d - abs(y - row)
    if x_range > 0:
        ranges = add_range(ranges, (x - x_range, x + x_range))

sum(y - x + 1 for x, y in ranges) - len(row_stations)

5403290

# Part 2

An inefficient (brute force) solution.

In [6]:
MIN_POS, MAX_POS = 0, 4000000

In [7]:
def scan_row(row, x0=MIN_POS, y0=MAX_POS):
    ranges = []
    for x, y, bx, by, d in sensors:
        x_range = d - abs(y - row)
        if x_range > 0:
            ranges = add_range(ranges, (x - x_range, x + x_range))

    if sum(min(y, y0) - max(x, x0) for x, y in ranges) < y0 - x0:
        dbx = max(y for _, y in ranges if y < y0) + 1
        return dbx*y0 + row

In [ ]:
"""
for row in range(4000001):
    tf = scan_row(row)
    if tf is not None:
        print(tf)
        break
"""

A slightly less inefficient linear programming solution.

In [8]:
import mip

In [9]:
def abs_value(model, x):
    HUGE = 1000000000
    absx = m.add_var(var_type=mip.INTEGER)
    sgn = m.add_var(var_type=mip.BINARY)
    model += absx <= x + HUGE * (1 - sgn)
    model += absx >= x - HUGE * (1 - sgn)
    model += absx <= -x + HUGE * sgn
    model += absx >= -x - HUGE * sgn
    return absx

m = mip.Model()
x = m.add_var("x", lb=MIN_POS, ub=MAX_POS, var_type=mip.INTEGER)
y = m.add_var("y", lb=MIN_POS, ub=MAX_POS, var_type=mip.INTEGER)
for sx, sy, *_, d in sensors:
    dx = abs_value(m, sx - x)
    dy = abs_value(m, sy - y)
    m += dx + dy >= d + 1
status = m.optimize()
tfx = int(x.x)
tfy = int(y.x)
tfx*MAX_POS + tfy

10291582906626

A more efficient solution based on the following property.  
The beacon location should be at the intersection of two 1-wide bands based on pairs of squares.

If the beacon is on the border of the considered area, this solution does not work.

In [10]:
downslope = []
upslope = []
for i, (x1, y1, *_, d1) in enumerate(sensors):
    for x2, y2, *_, d2 in sensors[:i]:
        if abs(x2 - x1) + abs(y2 - y1) == d2 + d1 + 2:
            assert x1 != x2 and y1 != y2, "Not supported"
            # compute the intersection with x = 0
            if (x1 < x2) == (y1 < y2):
                if x1 < x2:
                    o = y1 + x1 + d1 + 1
                else:
                    o = x2 + y2 + d2 + 1
                downslope.append(o)
            else:
                if x1 < x2:
                    o = y1 - x1 - d1 - 1
                else:
                    o = y2 - x2 - d2 - 1
                upslope.append(o)

In [11]:
def is_unknown(sensors, point):
    px, py = point
    for x, y, *_, d in sensors:
        if abs(x - px) + abs(y - py) <= d:
            return False
    return True

In [12]:
for u in upslope:
    for d in downslope:
        x = (d - u) // 2
        y = u + x
        assert u + x == d - x
        if is_unknown(sensors, (x, y)):
            print(f"Valid intersection at ({x}, {y}): tuning frequency is {x*4000000 + y}")
        else:
            print(f"Invalid intersection at ({x}, {y})")

Valid intersection at (2572895, 2906626): tuning frequency is 10291582906626
